In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms

In [3]:
mean = [0.49159187, 0.48234594, 0.44671956]
std = [0.23834434, 0.23486559, 0.25264624]

In [4]:
train_transform = transforms.Compose([
    # changing kernel
    # transforms.Resize(128)
    transforms.Resize(32),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [5]:
test_transform = transforms.Compose([
    # changing kernel
    # transforms.Resize(128)
    transforms.Resize(32),
    transforms.ToTensor(),
    transforms.Normalize(mean, std)
])

In [8]:
trainset = torchvision.datasets.CIFAR10(root='datasets/cifar10/train',
                                       train=True,
                                       download=True,
                                       transform=train_transform)

Files already downloaded and verified


In [9]:
testset = torchvision.datasets.CIFAR10(root='datasets/cifar10/train',
                                       train=False,
                                       download=True,
                                       transform=test_transform)

Files already downloaded and verified


In [10]:
trainloader = torch.utils.data.DataLoader(trainset,
                                         batch_size=16,
                                         shuffle=True,
                                         num_workers=2)

In [11]:
testloader = torch.utils.data.DataLoader(testset,
                                         batch_size=16,
                                         shuffle=False,
                                         num_workers=2)

In [12]:
class_names = trainset.classes
class_names

['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [13]:
img, label = iter(trainloader).next()

In [14]:
img.shape

torch.Size([16, 3, 32, 32])

In [15]:
label

tensor([5, 3, 9, 8, 3, 2, 9, 1, 9, 6, 1, 0, 5, 4, 5, 2])

In [16]:
in_size = 3

hid1_size = 16
hid2_size = 32
# changing kernel
# hid3_size = 64

out1_size = 40
# changing kernel
# out1_size = 2000
out2_size = 10

k_conv_size = 5
# k_conv_size = 9

In [30]:
class ConvNet(nn.Module):

  def __init__(self):
    super(ConvNet, self).__init__()

    self.layer1 = nn.Sequential(
        nn.Conv2d(in_size, hid1_size, k_conv_size),
        nn.BatchNorm2d(hid1_size),
        nn.ReLU(),
        # nn.Sigmoid(),
        # nn.Tanh(),
        # nn.ELU
        nn.MaxPool2d(kernel_size=2)
        # nn.AvgPool2d(kernel_size=2)
        # p = inf gives maxpooling, 0 gives average pooling
        # nn.LPPool2d(1, kernel_size=2)
    )

    self.layer2 = nn.Sequential(
        nn.Conv2d(hid1_size, hid2_size, k_conv_size),
        nn.BatchNorm2d(hid2_size),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.layer3 = nn.Sequential(
        nn.Linear(hid2_size * k_conv_size * k_conv_size, out1_size),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(out1_size, out2_size)
    )
#     self.layer4 = nn.Sequential(
#         nn.Linear(hid3_size * k_conv_size * k_conv_size, out1_size),
#         nn.ReLU(),
#         nn.Dropout(0.5),
#         nn.Linear(out1_size, out2_size)
#     )

    def forward(self, x):

        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)

        out = self.layer3(out)

        return F.log_softmax(out, dim=-1)

In [31]:
model = ConvNet()

In [32]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [33]:
model.to(device)

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=800, out_features=40, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=40, out_features=10, bias=True)
  )
)

In [34]:
learning_rate = 0.001
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [35]:
total_step = len(trainloader)
num_epochs = 10
loss_values = list()

In [40]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(trainloader):
        
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 2000 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, i+1, total_step, loss.item()))

            loss_values.append(loss.item())
print('Finished Training')

NotImplementedError: 

In [ ]:
x = range(1,11)

plt.figure(figsize=(12,10))
plt.plot(x, loss_values)
plt.xlabel('Step')
plt.ylabel('Loss')

In [ ]:
model.eval()

with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in testloader:
        
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    print('Accuracy of the model on the 10000 test images: {}%'.format(100*correct/total))

In [ ]:
sample_img, _ = testset[23]
sample_img.shape

In [ ]:
sample_img = np.transpose(sample_img, (1,2,0))

In [ ]:
m, M = sample_img.min(), sample_img.max()

sample_img = (1/(abs(m)*M))* sample_img + 0.5

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(sample_img)

In [ ]:
class_names[9]